###**Import Library yang Dibutuhkan untuk Menjalankan Model BERT**



In [ ]:
!git clone https://github.com/ezaaputra/indonlu.git
!pip install emoji
!pip install transformers

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import classification_report
from collections import defaultdict

#NLP
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import emoji

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

#Model IndoBERT
import random
import torch
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm

from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
from indonlu.utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader
from indonlu.utils.forward_fn import forward_sequence_classification
from indonlu.utils.metrics import document_sentiment_metrics_fn

Cloning into 'indonlu'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 460 (delta 119), reused 108 (delta 108), pack-reused 312
Receiving objects: 100% (460/460), 4.83 MiB | 8.21 MiB/s, done.
Resolving deltas: 100% (229/229), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 3.7 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


###**Import Dataset**

In [ ]:
from pandas import ExcelFile
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
df = pd.read_csv('/content/drive/My Drive/PROJECT/BDC (private)/BDC/DATASET/data (3).csv', sep=",")
df = df.dropna() #hapus NaN
df.isnull().sum()

df.head()
#df.describe
#df.groupby('label').describe().T

NameError: name 'pd' is not defined

In [ ]:
df_demografi = df[df['label']=='Demografi']
print("df_demografi.shape = ", df_demografi.shape)

df_ekonomi = df[df['label']=='Ekonomi']
print("df_ekonomi.shape = ", df_ekonomi.shape)

df_geografi = df[df['label']=='Geografi']
print("df_geografi.shape = ", df_geografi.shape)

df_ideologi = df[df['label']=='Ideologi']
print("df_ideologi.shape = ", df_ideologi.shape)

df_hankam = df[df['label']=='Pertahanan dan Keamanan']
print("df_hankam.shape = ", df_hankam.shape)

df_politik = df[df['label']=='Politik']
print("df_politik.shape = ", df_politik.shape)

df_sosbud = df[df['label']=='Sosial Budaya']
print("df_sosbud.shape = ", df_sosbud.shape)

df_sda = df[df['label']=='Sumber Daya Alam']
print("df_sda.shape = ", df_sda.shape)

df_demografi.shape =  (55, 2)
df_ekonomi.shape =  (268, 2)
df_geografi.shape =  (19, 2)
df_ideologi.shape =  (253, 2)
df_hankam.shape =  (257, 2)
df_politik.shape =  (2860, 2)
df_sosbud.shape =  (384, 2)
df_sda.shape =  (134, 2)


###**HANDLING IMBALANCED DATASET**

***Random Oversampling (Blind Copy)***

Buat semua data di setiap label menjadi sebanyak data terbanyak yakni data label politik sebanyak 2860 data

In [ ]:
#jumlah baris (observasi) dari masing-masing label
count_df_demografi = df_demografi.shape[0]
count_df_ekonomi = df_ekonomi.shape[0]
count_df_geografi = df_geografi.shape[0]
count_df_ideologi = df_ideologi.shape[0]
count_df_hankam = df_hankam.shape[0]
count_dF_politik = df_politik.shape[0]
count_df_sosbud = df_sosbud.shape[0]
count_df_sda = df_sda.shape[0]

In [ ]:
df_demografi_oversampled = df_demografi.sample(2860, replace = True)
df_ekonomi_oversampled = df_ekonomi.sample(2860, replace = True)
df_geografi_oversampled = df_geografi.sample(2860, replace = True)
df_ideologi_oversampled = df_ideologi.sample(2860, replace = True)
df_hankam_oversampled = df_hankam.sample(2860, replace = True)
df_sosbud_oversampled = df_sosbud.sample(2860, replace = True)
df_sda_oversampled = df_sda.sample(2860, replace = True)

#gabungkan seluruh dataset
df_randomover = pd.concat([df_demografi_oversampled, df_ekonomi_oversampled, df_geografi_oversampled, df_ideologi_oversampled, df_hankam_oversampled, df_politik, df_sosbud_oversampled, df_sda_oversampled], axis=0)
df_randomover.shape

print(df_randomover.label.value_counts())

label
Demografi                  2860
Ekonomi                    2860
Geografi                   2860
Ideologi                   2860
Pertahanan dan Keamanan    2860
Politik                    2860
Sosial Budaya              2860
Sumber Daya Alam           2860
Name: count, dtype: int64


***SMOTE***

Buat semua data di setiap label menjadi sebanyak data terbanyak yakni data label politik sebanyak 2861 data

In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

!pip install imbalanced-learn scikit-learn
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Dataset Politik dan Sosial Budaya
df_psb = pd.concat([df_sosbud, df_politik], axis=0)

#Dataset Politik dan Ekonomi
df_pe = pd.concat([df_ekonomi, df_politik], axis=0)

#Dataset Politik dan Hankam
df_ph = pd.concat([df_hankam, df_politik], axis=0)

#Dataset Politik dan Ideologi
df_pi = pd.concat([df_ideologi, df_politik], axis=0)

#Dataset Politik dan SDA
df_ps = pd.concat([df_sda, df_politik], axis=0)

#Dataset Politik dan Demografi
df_pd = pd.concat([df_demografi, df_politik], axis=0)

#Dataset Politik dan Geografi
df_pg = pd.concat([df_geografi, df_politik], axis=0)

In [ ]:
X = df_psb.drop('label', axis = 'columns')
y = df_psb['label']

y.value_counts()

label
Politik          2860
Sosial Budaya     384
Name: count, dtype: int64

SMOTE dirancang khusus untuk menangani imbalanced pada dataset numerik sehingga dataset ini perlu di preprocessing terlebih dahulu

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#konversi data teks ke vektor TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform

In [ ]:
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X_tfidf, y)

NameError: name 'SMOTE' is not defined

In [ ]:
# You might need to decode y_sm back to original labels later if needed
# y_decoded = encoder.inverse_transform(y_sm.reshape(-1, 1))

In [ ]:
y_sm.value_counts()

***Data Augmentation***



In [ ]:
from tqdm.auto import tqdm
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Dataset Politik dan Sosial Budaya
df_psb = pd.concat([df_sosbud, df_politik], axis=0)

#Dataset Politik dan Ekonomi
df_pe = pd.concat([df_ekonomi, df_politik], axis=0)

#Dataset Politik dan Hankam
df_ph = pd.concat([df_hankam, df_politik], axis=0)

#Dataset Politik dan Ideologi
df_pi = pd.concat([df_ideologi, df_politik], axis=0)

#Dataset Politik dan SDA
df_ps = pd.concat([df_sda, df_politik], axis=0)

#Dataset Politik dan Demografi
df_pd = pd.concat([df_demografi, df_politik], axis=0)

#Dataset Politik dan Geografi
df_pg = pd.concat([df_geografi, df_politik], axis=0)

In [ ]:
!pip install nlpaug
import nlpaug.augmenter.word.context_word_embs as aug
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 5.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
#tf_without_balancing = TfidfVectorizer()
#X_tf_wob = tf_without_balancing.fit_transform(df_pd['text'])
#X_tf_wob = X_tf_wob.toarray()

#X_tf_wob

In [ ]:
def augmentMyData(df, augmenter, repetitions = 1, samples = 200):
  augmented_texts = []
  #select only the minority class samples
  spam_df = df[df['label'] == 1].reset_index(drop=True) #removes unnecessary index column
  for i in tqdm(np.random.randint(0, len(spam_df), samples)):
    #generating 'n_samples' augemented texts
    for _ in range(repetitions):
      augmented_text = augmenter.augment(spam_df['text'].iloc[i])
      augmented_texts.append(augmented_text)

  data = {
      'label' : 1,
      'text' : augmented_texts
  }

  aug_df = pd.DataFrame(data)
  df = shuffle(pd.concat([df, aug_df]).reset_index(drop=True))
  return df

*Untuk Kelas Politik dan Demografi (df_pd)*

In [ ]:
label_map_pd = {
    'Politik': 0,
    'Demografi': 1,
}

df_pd['label'] = df_pd['label'].map(label_map_pd)
df_pd.head()

,text,label
2,dukung dukung ridwan kamil skema balik mayorit...,1
60,konsisten suara lawan radikalisme intoleransi ...,1
333,program kuliah gratis anak tentara polisi anak...,1
463,jumat agenda pranowomahfud kampanye jawa barat...,1
584,gaya campaign sejuk masbowogbran okegas tingka...,1


In [ ]:
tf_without_balancing_pd = TfidfVectorizer()
X_tf_wob_pd = tf_without_balancing_pd.fit_transform(df_pd['text'])
X_tf_wob_pd = X_tf_wob_pd.toarray()

X_tf_wob_pd

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
sample_text_pd = df_pd['text'].iloc[100]
sample_text_pd

'referensi jkw ajar rakyat tolol janji kembali tanah bilang'

In [ ]:
augmented_sample_text_pd = augmenter.augment(sample_text_pd)
augmented_sample_text_pd

['yang referensi sa jkw sa ajar rakyat tolol pang janji bang kembali lang tanah dan bilang']

In [ ]:
aug_df_pd = augmentMyData(df_pd, augmenter, samples = 2860)

  0%|          | 0/2860 [00:00<?, ?it/s]

KeyboardInterrupt: 

*Untuk Kelas Politik dan Sosial Budaya (df_psb)*

In [ ]:
label_map_psb = {
    'Politik': 0,
    'Sosial Budaya': 1,
}

df_psb['label'] = df_psb['label'].map(label_map_psb)
df_psb.head()

,text,label
18,angkat martabat tani terus terima bansos hasil...,1
25,sesuai selera masyarakat sumatera barat,1
36,post has reached likes and on dialog kadin tvo...,1
38,kejam warga situ sedia bangun jis janji memabn...,1
98,video bawah ambil bukti rembug rutin bukti lib...,1


In [ ]:
tf_without_balancing_pd = TfidfVectorizer()
X_tf_wob_pd = tf_without_balancing_pd.fit_transform(df_pd['text'])
X_tf_wob_pd = X_tf_wob_pd.toarray()

X_tf_wob_pd

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
sample_text_psb = df_psb['text'].iloc[100]
sample_text_psb

'fanart manies hehehehe biar dukung pakai ai fanartanies'

In [ ]:
augmented_sample_text_psb = augmenter.augment(sample_text_psb)
augmented_sample_text_psb

['fanart so manies a hehehehe biar ta dukung a pakai to ai and fanartanies']

In [ ]:
aug_df_psb = augmentMyData(df_psb, augmenter, samples = 2860)

  0%|          | 0/2860 [00:00<?, ?it/s]

*Untuk Kelas Politik dan Ekonomi (df_pe)*

In [ ]:
label_map_pe = {
    'Politik': 0,
    'Ekonomi': 1,
}

df_pe['label'] = df_pe['label'].map(label_map_pe)
df_pe.head()

,text,label
13,dapat apbd jumbo indonesia capai rp,1
34,pasang duet cawapres motor ubah kredit koprasi...,1
39,terap program lancar ekonomi biru internasiona...,1
61,koneksi internet butuh pokok masyarakat jalan ...,1
62,jaua suparna si fakultas ekonomi si bijak uang...,1


In [ ]:
#tf_without_balancing_pd = TfidfVectorizer()
#X_tf_wob_pd = tf_without_balancing_pd.fit_transform(df_pd['text'])
#X_tf_wob_pd = X_tf_wob_pd.toarray()

#X_tf_wob_pd

In [ ]:
sample_text_pe = df_pe['text'].iloc[100]
sample_text_pe

'pasang cawapres ri ekonomi tumbuh capai pesat'

In [ ]:
augmented_sample_text_pe = augmenter.augment(sample_text_pe)
augmented_sample_text_pe

['sa pasang cawapres indonesia ri ekonomi malay tumbuh capai di pesat']

In [ ]:
aug_df_pe = augmentMyData(df_pe, augmenter, samples = 2860)

  0%|          | 0/2860 [00:00<?, ?it/s]

*Untuk Kelas Politik dan Hankam (df_ph)*

In [ ]:
label_map_ph = {
    'Politik': 0,
    'Pertahanan dan Keamanan': 1,
}

df_ph['label'] = df_ph['label'].map(label_map_ph)
df_ph.head()

,text,label
6,kemenhan tim satgas air unhan kaji bantu air s...,1
10,pranowomahfud tekan siap butuh dasar alutsista...,1
11,calon nomor urut mampu hadap debat tiga debat ...,1
37,according gaza is opressed because they dont h...,1
40,viral ancam tembak tim hukum tempuh jalur hukum,1


In [ ]:
#tf_without_balancing_pd = TfidfVectorizer()
#X_tf_wob_pd = tf_without_balancing_pd.fit_transform(df_pd['text'])
#X_tf_wob_pd = X_tf_wob_pd.toarray()

#X_tf_wob_pd

In [ ]:
sample_text_ph = df_ph['text'].iloc[100]
sample_text_ph

'terima kasih menteri tahan dengar aspirasi perihal sejahtera prajurit tentara tentara garda depan tahan bangsa sejahtera hidup tenang jalan tugas'

In [ ]:
augmented_sample_text_ph = augmenter.augment(sample_text_ph)
augmented_sample_text_ph

['the terima masjid kasih palace menteri tahan dengar aspirasi hotel perihal sejahtera air prajurit tentara tentara air garda depan java tahan batavia bangsa sejahtera indonesia hidup tenang jalan tugas']

In [ ]:
aug_df_ph = augmentMyData(df_ph, augmenter, samples = 2000)

NameError: name 'df_ph' is not defined

***Combination (Oversampling + Undersampling)***

***Class Weighting***

###**Split Dataset**

*   Data Training 70%
*   Data Validasi 20%
*   Data Training 10%

In [ ]:
#Train Val Split
train_set, val_set = train_test_split(df, test_size = 0.3, stratify = df.label, random_state = 1)
val_set, test_set = train_test_split(val_set, test_size = 0.33, stratify = val_set.label, random_state = 1)

In [ ]:
print(f'Train shape: {train_set.shape}')
print(f'Val shape: {val_set.shape}')
print(f'Test shape: {test_set.shape}')

Train shape: (2961, 2)
Val shape: (850, 2)
Test shape: (420, 2)


In [ ]:
#export to tsv
train_set.to_csv('train_set.tsv', sep ='\t', header = None, index = False)
val_set.to_csv('val_set.tsv', sep='\t', header = None, index = False)
test_set.to_csv('test_set.tsv', sep='\t', header = None, index = False)

###**Tahap Training Dataset**

In [ ]:
#common functions
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

def count_param(module, trainable = False):
  if trainable:
    return sum (p.numel() for p in module.parameters() if p.requires_grad)
  else:
    return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
  for param_group in optimizer.param_groups:
    return param_group['lr']

def metrics_to_string(metric_dict):
  string_list = []
  for key, value in metric_dict.items():
    string_list.append('{}:{:.2f}'.format(key,value))
  return ' '.join(string_list)

#set random seed
set_seed(27)
print(set_seed)

<function set_seed at 0x79e0ed3dfeb0>


*Tokenisasi Dataset dan Instalasi Pretrained Model IndoBERT*

In [ ]:
#Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS

#Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

#Struktur model
print(model)

#Jumlah parameter
print(count_param(model))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

*Data Loading*

In [ ]:
train_dataset_path = '/content/train_set.tsv'
valid_dataset_path = '/content/val_set.tsv'
test_dataset_path = '/content/test_set.tsv'

#fungsi dataset loader dari utils IndoNLU
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase = True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowecase = True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase = True)

train_loader = DocumentSentimentDataset(dataset = train_dataset, max_seq_len = 512, batch_size = 32, num_workers = 16, shuffle = True)
valid_loader = DocumentSentimentDataset(dataset = valid_dataset, max_seq_len = 512, batch_size = 32, num_workers = 16, shuffle = False)
test_loader = DocumentSentimentDataset(dataset = test_dataset, max_seq_len = 512, batch_size = 32, num_workers = 16, shuffle = False)

w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i) #word to index
print(iw2) #index to word


NameError: name 'DocumentSentimentDataset' is not defined

In [ ]:
train_df = pd.read_csv(train_dataset_path, sep='\t')
print(train_df.head())

In [ ]:
train_dataset_path = '/content/train_set.tsv'
data = pd.read_csv(train_dataset_path, sep='\t')
print(data.head(50))


In [ ]:
print("Hello World!")

Hello World!
